In [1]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")

# LLM Chain

In [2]:
llm.invoke("Did Greece land troops in Anatolia region in Turkey?")

"A sensitive topic!\n\nYou're likely referring to the Greco-Turkish War (1920-1922) and the Turkish War of Independence. During this period, Greece did indeed launch military campaigns into Anatolia, which is now part of modern-day Turkey.\n\nThe war was fought between Greece and Turkey over control of western Anatolia and eastern Macedonia. The conflict began in 1919, shortly after World War I ended, and lasted until 1922.\n\nIn September 1920, Greek forces entered western Anatolia, aiming to capture the region's key cities, including Smyrna (now İzmir) and Bursa. However, they faced fierce resistance from Turkish nationalist forces led by Mustafa Kemal Atatürk, who would later become Turkey's founding president.\n\nThe Greeks ultimately suffered a series of defeats, and their armies were forced to withdraw from Anatolia. The Treaty of Lausanne in 1923 officially ended the war, leading to the exchange of populations between Greece and Turkey: some 1 million Greeks left Turkey for Gree

In [3]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [4]:
chain = prompt | llm 

In [5]:
chain.invoke({"input": "how can langsmith help with testing?"})

"As a world-class technical documentation writer, I'm excited to share some ideas on how Langsmith can assist with testing.\n\nLangsmith's language processing capabilities can be leveraged in various ways to support testing efforts:\n\n1. **Automated Testing**: Langsmith can generate test scripts or test cases based on the text it processes. For instance, if you provide a set of requirements documents, Langsmith can create automated test cases that cover each requirement.\n2. **Test Data Generation**: Langsmith can help generate test data by processing natural language descriptions of test scenarios and generating relevant input values for testing purposes.\n3. **Error Message Analysis**: When errors occur during testing, Langsmith can assist in analyzing error messages to identify the root cause of the issue. It can also provide insights on how to reproduce the error or suggest potential solutions.\n4. **Test Report Generation**: Langsmith can help generate test reports by processing 

In [6]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [7]:
chain = prompt | llm | output_parser

In [8]:
chain.invoke({"input": "how can langsmith help with testing?"})

"As a world-class technical documentation writer, I'm excited to share some ideas on how Langsmith can assist with testing!\n\nLangsmith, being a powerful language processing AI, can be leveraged in various ways to streamline and enhance the testing process. Here are some potential applications:\n\n1. **Automated Testing**: Langsmith can generate test cases based on natural language descriptions of expected behavior. For instance, you could write a brief description of a specific scenario or user story, and Langsmith would translate that into a set of automated test scripts.\n2. **Error Message Generation**: When errors occur during testing, Langsmith can help create error messages that are clear, concise, and easy to understand. This can improve the overall quality of your testing process by reducing misunderstandings and misinterpretations.\n3. **Test Data Generation**: Langsmith can generate test data based on specific parameters or scenarios, which can be useful for regression test

# Retrieval Chain

In [9]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

In [10]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model='llama3')

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
<context>
  {context}
</context>
Question: {input}""")
document_chain = create_stuff_documents_chain(llm, prompt)

In [13]:
from langchain_core.documents import Document
document_chain.invoke({
   "input": "how can langsmith help with testing?",
   "context": [Document(page_content="langsmith can let you visualize test results")]
})

'According to the given context, LangSmith can "let you visualize test results". This implies that LangSmith is a tool or service that enables users to see and understand the results of tests in a more interactive and visual manner.'

In [14]:
from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [15]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

According to the provided context, LangSmith can help with testing in several ways:

1. **Debugging**: By having tracing enabled by default, developers can quickly understand how their model is performing and debug where it's failing.
2. **Test-driven approach**: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.
3. **Comparison View**: The platform provides a comparison view for test runs, allowing developers to track and diagnose regressions in test scores across multiple revisions of their application.
4. **Playground**: LangSmith's playground environment allows rapid iteration and experimentation with different prompts and models.
5. **Beta Testing**: Langsmith supports collecting more data on how the LLM applications are performing in real-world scenarios, including gathering human feedback on responses.
6. **Annotating Traces**: The platform enables annotators to closely insp

# Conversation Retrieval Chain

In [16]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
   MessagesPlaceholder(variable_name="chat_history"),
   ("user", "{input}"),
   ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [17]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2024 LangChain, Inc.', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them somethin

In [18]:
prompt = ChatPromptTemplate.from_messages([
   ("system", "Answer the user's questions based on the below context:\n\n{context}"),
   MessagesPlaceholder(variable_name="chat_history"),
   ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [19]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
   "chat_history": chat_history,
   "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2024 LangChain, Inc.', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fi

 # Agent

In [20]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
   retriever,
   "langsmith_search",
   "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)